In [3]:
import boto3
from datetime import datetime
import pandas as pd

In [14]:
client = 'trio'
env = 'dev'
# facilities = [1,3,4,5,6,7,8,9,10,13,21] avante
facilities = [279]  # trio
CACHED_S3_BUCKET = 'saiva-dev-data-bucket'
task_name = 'saiva-3-day-hosp-v2'
prediction_start_date = '2020-12-01'
prediction_end_date = '2021-01-31'

JOB_QUEUE = 'dev-batch-queue'
JOB_DEFINITION = 'dev-saiva-3-day-hosp-v3'

saiva = boto3.session.Session()
boto3.setup_default_session()

In [4]:
vpn_instance = {
    "dev": "i-07197a52885c34d68",
    "staging": "i-07d9c5587e7637960",
    "prod": "i-0e5f01117bb50fdb1"
}

AutoScalingGroupName = {
    "dev": "dev-Internal-ASG",
    "staging": "staging-Internal-ASG",
    "prod": "prod-Internal-ASG"
}

ClusterName = {
    "dev": "dev-Internal",
    "staging": "staging-Internal",
    "prod": "prod-Internal"  
}

networkConfiguration = {
    "dev": {
        'awsvpcConfiguration': {
            'securityGroups': ['sg-008159c0210f49f96'], 
            'subnets': ['subnet-0eb41526eff222cab', 'subnet-04b6e7f553bd31909'], 
            'assignPublicIp': 'DISABLED'}},
    "staging": {
        'awsvpcConfiguration': {
            'securityGroups': ['sg-0f8d382201750604e'], 
            'subnets': ['subnet-0b9343ae7c5712608', 'subnet-01d3901613027f4d9'], 
            'assignPublicIp': 'DISABLED'}},
    "prod": {
            "awsvpcConfiguration": {
                "securityGroups": ["sg-041a9f4c8e2f4fd56"],
                "subnets": ["subnet-0c87ee56c3e4e27d1", "subnet-065f42df2509bd9f5"],
                "assignPublicIp": "DISABLED",
            }
        }
}

In [5]:
aws_batch_client = boto3.client('batch', region_name='us-east-1')

In [18]:
response = aws_batch_client.list_jobs(
    jobQueue=JOB_QUEUE,
    maxResults=100,
)
response

{'ResponseMetadata': {'RequestId': 'c9111e57-b1ae-4bc5-8bcd-993929c50923',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 10 May 2021 09:11:42 GMT',
   'content-type': 'application/json',
   'content-length': '21',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c9111e57-b1ae-4bc5-8bcd-993929c50923',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'fGvQOGiBIAMFStg=',
   'access-control-expose-headers': 'X-amzn-errortype,X-amzn-requestid,X-amzn-errormessage,X-amzn-trace-id,X-amz-apigw-id,date',
   'x-amzn-trace-id': 'Root=1-6098f8ce-18f365fe570dbe2369004e91'},
  'RetryAttempts': 0},
 'jobSummaryList': []}

In [17]:
def start_backfill(facility_id, prediction_start_date,prediction_end_date):
    response = aws_batch_client.submit_job(
        jobName='job1',
        jobQueue=JOB_QUEUE,
        jobDefinition=JOB_DEFINITION,
        containerOverrides={
            'command': [
                "/bin/bash",
                "-c",
                f"python /src/run_backfill.py --client {client} --facilityids '[{facility_id}]' --prediction_start_date '{prediction_start_date}' --prediction_end_date '{prediction_end_date}' --s3-bucket {CACHED_S3_BUCKET} --replace_existing_predictions True ",    
            ],
        }
    )
    return response

In [19]:
print(datetime.now())
results = [start_backfill(
    facility_id=facility, 
    prediction_start_date=prediction_start_date, 
    prediction_end_date=prediction_end_date) for facility in facilities]
results

2021-05-10 09:11:46.785693


[{'ResponseMetadata': {'RequestId': '332b0399-130e-4ff2-95cc-aa1a750af38c',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Mon, 10 May 2021 09:11:46 GMT',
    'content-type': 'application/json',
    'content-length': '154',
    'connection': 'keep-alive',
    'x-amzn-requestid': '332b0399-130e-4ff2-95cc-aa1a750af38c',
    'access-control-allow-origin': '*',
    'x-amz-apigw-id': 'fGvQ8FkcIAMFwIw=',
    'access-control-expose-headers': 'X-amzn-errortype,X-amzn-requestid,X-amzn-errormessage,X-amzn-trace-id,X-amz-apigw-id,date',
    'x-amzn-trace-id': 'Root=1-6098f8d2-64ea98ce49d0595b3a885f51'},
   'RetryAttempts': 0},
  'jobName': 'job1',
  'jobId': '75db229a-d100-4077-8174-621f32a33200'}]